# Problem Statement

The `fast.ai` library has a callback to track training metrics history. However, the history is reported via console, or Jupyter widget, and there are no callbacks to store these results into CSV format. In this notebook, the author proposes his approach to implement a callback similar to [CSVLogger from Keras library](https://github.com/keras-team/keras/blob/master/keras/callbacks.py#L1135) which will save tracked metrics into persistent file.

In [1]:
%reload_ext autoreload

In [2]:
%autoreload 2

In [47]:
from fastai import *
from fastai.torch_core import *
from fastai.vision import *
from fastai.metrics import *
from torchvision.models import resnet18

In [53]:
@dataclass
class CSVLogger(LearnerCallback):
    "A `LearnerCallback` that "
    filename:str='history.csv'

    def __post_init__(self):
        self.path = Path(self.filename)
        self.file = None

    @property
    def header(self):
        return self.learn.recorder.names
    
    def read_logged_file(self):
        return pd.read_csv(self.path)

    def on_train_begin(self, metrics_names:StrList, **kwargs:Any)->None:
        self.path.parent.mkdir(parents=True, exist_ok=True)
        self.file = self.path.open('w')
        self.file.write(','.join(self.header) + '\n')

    def on_epoch_end(self, epoch:int, smooth_loss:Tensor, last_metrics:MetricsList, **kwargs:Any)->bool:
        self.write_stats([epoch, smooth_loss] + last_metrics)

    def on_train_end(self, **kwargs:Any)->None:
        self.file.flush()
        self.file.close()

    def write_stats(self, stats:TensorOrNumList)->None:
        stats = [str(stat) if isinstance(stat, int) else f'{stat:.6f}'
                 for name,stat in zip(self.header,stats)]
        str_stats = ','.join(stats)
        self.file.write(str_stats + '\n')

## Example

Let's train MNIST classifier and track its metrics. All the metrics listed in `metrics` array, and also epoch number, train and valid loss should be saved into file. Then we can read this file and process somehow.  

In [66]:
path = untar_data(URLs.MNIST_TINY)

In [67]:
data = ImageDataBunch.from_folder(path)

In [68]:
learn = ConvLearner(data, resnet18, metrics=[accuracy, error_rate])

In [69]:
cb = CSVLogger(learn)

In [70]:
learn.fit(3, callbacks=[cb])

Total time: 00:02
epoch  train loss  valid loss  accuracy  error_rate
1      0.463691    0.321506    0.889843  0.110157    (00:00)
2      0.291690    0.281163    0.876967  0.123033    (00:00)
3      0.269455    0.289067    0.879828  0.120172    (00:00)



In [71]:
log_df = cb.read_logged_file()
log_df

,epoch,train loss,valid loss,accuracy,error_rate
0,1,0.463691,0.321506,0.889843,0.110157
1,2,0.291690,0.281163,0.876967,0.123033
2,3,0.269455,0.289067,0.879828,0.120172


## Tests

The tests are duplicated in [test_logger.py](./test_logger.py) file and could be invoked with command:
```bash
$ python -m pytest test_logger.py
```

### Case 1: Callback has required properties and default values after initialization

In [72]:
cb = CSVLogger(mock.Mock(), filename=history)

assert cb.filename
assert not cb.path.exists()
assert cb.file is None

NameError: name 'mock' is not defined

In [ ]:
assert cb.path.exists()
assert not log_df.empty
assert learn.recorder.names == log_df.columns.tolist()